This notebook will make 'virtual multi bat' audio files. I use the word 'virtual' here sensu Obrist 1995, Ratcliffe et al. 2004, and Fawcett & Ratcliffe 2015. In the previous experiments the 'virtual' refers to the fact that the authors are comparing the actual difference between acoustic measures when alone vs when they are together, or the difference in call parameters when alone vs in groups. 

Unlike in previous studies which calculated the *difference* in the acoustic measures across single and multi bat contexts, here we will mix the audio data and the generate measures. This raw audio will then be used for analysis. In the end, the observed multi bat audio will be compared to the virtual multi bat audio. If the parameters in the observed and multi bat audio are similar, this is strong evidence that there isn't much of a change in the echolocation behaviour. If there is a difference in the observed multi bat and virtual multi bat, the parameter with the most change is actually informative. 

### Where are the 'single audio' files from 
In a previous notebook ('analysis/Resampling the measurements.ipynb'), I actually ran a comparison of single vs multi bat annotation audio parameters. To perform this analysis I relied on measurements of the annotation audio with 50ms split windows set of valid single and multi annotation audio files ('non_silent_measurements_20dBthreshold.csv'). I will use the single calls in this 'valid set' to generate the virtual multi bat audio.


### How will the virtual audio files be made

The WAV files will be loaded and added directly without any further processing. I will use the high-passed audio to reduce the amount of processing I will have to do. I will *not* do any kind of normalisation because the entire point of this exercise is to see if parameters like amplitude will increase or decrease in multi-bat situations. 

Ideally each multi bat audio file will be matched by a virtual multi bat file of the same length. This will help in keeping things the same eg. # of audio segments that will originate from the virtual audio file vs from the observed audio file. 


#### Making interval-separated virtual audio files 
The matched audio files come from annotations that are often made very close to each other in time. This is because bat activity is autocorrelated in time. It may even be that the same bat flies in and out over short periods of time. To reduce this we will create virtual multi-bat files from single bat files that are spaced at least 5 minutes from each other. This will reduce the amount of pseudo-replication seen in our virtual multi-bat audio files. If we do not perform this step, many of the virtual multi-bat audio files may actually have audio recordings from the same bat added together - resulting in an underestimation of the variation in the data.


- Author: Thejasvi Beleyur
- Date: 2020-07-20


### References
- Obrist, M. K. (1995). Flexible bat echolocation: the influence of individual, habitat and conspecifics on sonar signal design. Behavioral ecology and sociobiology, 36(3), 207-219.
- Ratcliffe, J. M., Hofstede, H. M. T., Avila-Flores, R., Fenton, M. B., McCracken, G. F., Biscardi, S., ... & Spanjer, G. (2004). Conspecifics influence call design in the Brazilian free-tailed bat, Tadarida brasiliensis. Canadian Journal of Zoology, 82(6), 966-971.
- Fawcett, K., & Ratcliffe, J. M. (2015). Clutter and conspecifics: a comparison of their influence on echolocation and flight behaviour in Daubenton’s bat, Myotis daubentonii. Journal of Comparative Physiology A, 201(3), 295-304.

In [1]:
%matplotlib notebook


In [2]:
import datetime as dt
import glob
import os
import random
random.seed(82320)
import sys
sys.path.append('../correct_call_annotations/')
sys.path.append('../')
sys.path.append('../combined_analysis/')
sys.path.append('../individual_call_analysis/analysis/')

import correct_call_annotations.correct_call_annotations as cca

import reduce_pseudorep
from reduce_pseudorep import annots_by_time
import format_and_clean 
from format_and_clean import ind_call_format as icf


import soundfile as sf
import tqdm
import numpy as np 
import pandas as pd
from pandas.core.common import flatten
import matplotlib.pyplot as plt 
plt.rcParams['agg.path.chunksize'] = 10000

In [3]:
%matplotlib notebook

In [4]:
print(f'Notebook run started at {dt.datetime.now()}')

Notebook run started at 2020-12-17 15:16:08.565788


In [5]:
verified_annotations = pd.read_csv('verified_annotations.csv')

In [6]:
np.unique(verified_annotations['num_bats'], return_counts=True)

(array([1, 2, 3, 4], dtype=int64), array([302,  86,  20,   6], dtype=int64))

### Getting annotation time stamps. 


In [7]:
video_annotations_folder = '../whole_data_analysis/annotations/corrected_HBC_video_annotations_Aditya/'
verified_annotations['annot_end_time'] = icf.get_startstoptime_from_annotation_id(verified_annotations['video_annot_id'],
                                                                             video_annotations_folder,
                                                                                 endtime=True)
verified_annotations['annot_start_time'] = icf.get_startstoptime_from_annotation_id(verified_annotations['video_annot_id'],
                                                                             video_annotations_folder,
                                                                                 endtime=False)

In [8]:
verified_annotations['posix_end_time'] = verified_annotations['annot_end_time'].apply(icf.convert_annotation_time_to_posix)
verified_annotations['posix_start_time'] = verified_annotations['annot_start_time'].apply(icf.convert_annotation_time_to_posix)

In [9]:
# sort all the annotations according to their posix start time 
verified_annotations = verified_annotations.sort_values(by='posix_start_time')

In [10]:
verified_annotations

,Unnamed: 0,valid_annotations,video_annot_id,num_bats,annot_end_time,annot_start_time,posix_end_time,posix_start_time
0,0,matching_annotaudio_Aditya_2018-08-16_21502300_18,Aditya_2018-08-16_21502300_18,1,2018-08-16 22:24:26,2018-08-16 22:24:25,1.534451e+09,1.534451e+09
1,1,matching_annotaudio_Aditya_2018-08-16_21502300_20,Aditya_2018-08-16_21502300_20,1,2018-08-16 22:25:24,2018-08-16 22:25:21,1.534451e+09,1.534451e+09
2,2,matching_annotaudio_Aditya_2018-08-16_21502300_22,Aditya_2018-08-16_21502300_22,1,2018-08-16 22:25:47,2018-08-16 22:25:44,1.534451e+09,1.534451e+09
3,3,matching_annotaudio_Aditya_2018-08-16_21502300_24,Aditya_2018-08-16_21502300_24,1,2018-08-16 22:28:17,2018-08-16 22:28:16,1.534451e+09,1.534451e+09
4,4,matching_annotaudio_Aditya_2018-08-16_21502300_25,Aditya_2018-08-16_21502300_25,1,2018-08-16 22:28:47,2018-08-16 22:28:46,1.534451e+09,1.534451e+09
...,...,...,...,...,...,...,...,...
409,430,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_84,1,2018-08-20 03:35:29,2018-08-20 03:35:29,1.534729e+09,1.534729e+09
410,431,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_85,1,2018-08-20 03:35:44,2018-08-20 03:35:43,1.534729e+09,1.534729e+09
411,432,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_90,1,2018-08-20 03:37:58,2018-08-20 03:37:58,1.534729e+09,1.534729e+09
412,433,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_91,1,2018-08-20 03:38:01,2018-08-20 03:38:00,1.534729e+09,1.534729e+09


### Get the total number of single and multi bat annotation files

In [11]:
multi_bat_files = [verified_annotations[verified_annotations['num_bats']==each] for each in [2,3,4]]
pairs, triplets, quadruplets = multi_bat_files

singles = verified_annotations[verified_annotations['num_bats']==1].reset_index(drop=True)
multibats = pd.concat(multi_bat_files).reset_index(drop=True)

num_pairs = len(np.unique(pairs['video_annot_id']))
num_triplets = len(np.unique(triplets['video_annot_id']))
num_quadruplets = len(np.unique(quadruplets['video_annot_id']))
num_singles = len(np.unique(singles['video_annot_id']))

total_single_calls_needed = num_pairs*2 + num_triplets*3 + num_quadruplets*4
print(num_singles, num_pairs, num_triplets, num_quadruplets, total_single_calls_needed)


302 86 20 6 256


### There are sufficient single annotation audio files to make all required virtual multi bat audio files *uniquely*!
There are right now 361 single audio files, while, we need only a total of 256 files. This is good news in that there will be no need to repeat any files. 

In [12]:
verified_annotations

,Unnamed: 0,valid_annotations,video_annot_id,num_bats,annot_end_time,annot_start_time,posix_end_time,posix_start_time
0,0,matching_annotaudio_Aditya_2018-08-16_21502300_18,Aditya_2018-08-16_21502300_18,1,2018-08-16 22:24:26,2018-08-16 22:24:25,1.534451e+09,1.534451e+09
1,1,matching_annotaudio_Aditya_2018-08-16_21502300_20,Aditya_2018-08-16_21502300_20,1,2018-08-16 22:25:24,2018-08-16 22:25:21,1.534451e+09,1.534451e+09
2,2,matching_annotaudio_Aditya_2018-08-16_21502300_22,Aditya_2018-08-16_21502300_22,1,2018-08-16 22:25:47,2018-08-16 22:25:44,1.534451e+09,1.534451e+09
3,3,matching_annotaudio_Aditya_2018-08-16_21502300_24,Aditya_2018-08-16_21502300_24,1,2018-08-16 22:28:17,2018-08-16 22:28:16,1.534451e+09,1.534451e+09
4,4,matching_annotaudio_Aditya_2018-08-16_21502300_25,Aditya_2018-08-16_21502300_25,1,2018-08-16 22:28:47,2018-08-16 22:28:46,1.534451e+09,1.534451e+09
...,...,...,...,...,...,...,...,...
409,430,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_84,1,2018-08-20 03:35:29,2018-08-20 03:35:29,1.534729e+09,1.534729e+09
410,431,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_85,1,2018-08-20 03:35:44,2018-08-20 03:35:43,1.534729e+09,1.534729e+09
411,432,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_90,1,2018-08-20 03:37:58,2018-08-20 03:37:58,1.534729e+09,1.534729e+09
412,433,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_91,1,2018-08-20 03:38:01,2018-08-20 03:38:00,1.534729e+09,1.534729e+09


### How long are the observed multi bat audio files ?
The duration of the multi bat audio files are important because the duration decides how many audio segments can maximally be part of the analysed dataset. Of course, the actual number of segments that are above the threshold will decrease this. It still helps to reduce the disparity between the virtual and observed multi bat audio files.

In [13]:
pairs

,Unnamed: 0,valid_annotations,video_annot_id,num_bats,annot_end_time,annot_start_time,posix_end_time,posix_start_time
9,9,matching_annotaudio_Aditya_2018-08-16_21502300_36,Aditya_2018-08-16_21502300_36,2,2018-08-16 22:37:46,2018-08-16 22:37:39,1.534452e+09,1.534452e+09
10,10,matching_annotaudio_Aditya_2018-08-16_21502300_37,Aditya_2018-08-16_21502300_37,2,2018-08-16 22:37:46,2018-08-16 22:37:39,1.534452e+09,1.534452e+09
11,11,matching_annotaudio_Aditya_2018-08-16_21502300_39,Aditya_2018-08-16_21502300_39,2,2018-08-16 22:37:50,2018-08-16 22:37:47,1.534452e+09,1.534452e+09
12,12,matching_annotaudio_Aditya_2018-08-16_21502300_40,Aditya_2018-08-16_21502300_40,2,2018-08-16 22:37:52,2018-08-16 22:37:48,1.534452e+09,1.534452e+09
13,13,matching_annotaudio_Aditya_2018-08-16_21502300_43,Aditya_2018-08-16_21502300_43,2,2018-08-16 22:38:07,2018-08-16 22:38:05,1.534452e+09,1.534452e+09
...,...,...,...,...,...,...,...,...
391,410,matching_annotaudio_Aditya_2018-08-20_0200-030...,Aditya_2018-08-20_0200-0300_66,2,2018-08-20 02:44:51,2018-08-20 02:44:49,1.534726e+09,1.534726e+09
394,413,matching_annotaudio_Aditya_2018-08-20_0200-030...,Aditya_2018-08-20_0200-0300_73,2,2018-08-20 02:45:45,2018-08-20 02:45:44,1.534726e+09,1.534726e+09
396,415,matching_annotaudio_Aditya_2018-08-20_0200-030...,Aditya_2018-08-20_0200-0300_78,2,2018-08-20 02:50:08,2018-08-20 02:50:07,1.534726e+09,1.534726e+09
401,420,matching_annotaudio_Aditya_2018-08-20_0200-030...,Aditya_2018-08-20_0200-0300_92,2,2018-08-20 02:58:49,2018-08-20 02:58:42,1.534727e+09,1.534727e+09


In [14]:
multi_bat = []
for each in multi_bat_files:
    for file in each['valid_annotations']:
        multi_bat.append(file+'_hp.WAV')

In [15]:
audio_folder = '../individual_call_analysis/hp_annotation_audio/'
file_durations = []
for file_name in tqdm.tqdm(multi_bat):        
    file_path = cca.find_file_in_folder(file_name, audio_folder)
    file_durations.append(sf.info(file_path[0]).duration)
    

  0%|                                                                                          | 0/112 [00:00<?, ?it/s]

Match found!


  3%|██▏                                                                               | 3/112 [00:00<00:04, 26.17it/s]

Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!


  7%|█████▊                                                                            | 8/112 [00:00<00:03, 30.06it/s]

...

 96%|█████████████████████████████████████████████████████████████████████████████▏  | 108/112 [00:02<00:00, 37.63it/s]

Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!


100%|████████████████████████████████████████████████████████████████████████████████| 112/112 [00:03<00:00, 35.13it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 112/112 [00:03<00:00, 36.65it/s]

Match found!
Match found!


In [16]:
plt.figure()
plt.boxplot(file_durations);
plt.ylabel('Multi bat audio duration, s', fontsize=12)
plt.hlines(np.percentile(file_durations, [0,100]), 0.75, 1.25)
plt.text( 1.15, np.percentile(file_durations, [50]),f'median: {np.percentile(file_durations, 50)}',)
plt.text( 1.15, np.percentile(file_durations, [99.5])+0.1,f'max: {np.percentile(file_durations, 100)}',)
plt.text( 1.15, np.percentile(file_durations, [0])+0.1,f'minimum: {np.percentile(file_durations, 0)}',)

<IPython.core.display.Javascript object>

Text(1.15, [0.22], 'minimum: 0.12')

There is some more thought to be put into how to make virtual multi bat audio files. There may be situations, especially when adding a long-ish file with a short-ish file, where the addition of the two files doesn't really change anything because the calls in one file and the other don't overlap as the active call regions don't overlap. To prevent such cases, it is important to create virtual files that are *matched* with each other in duration, and more importantly, matched with an observed multi bat file in duration. The following set of steps will be followed while creating a virtual multi bat file:

1. Choose one multi bat audio file and get its duration, $multibat_{duration}$
1. Generate a time-interval separated list of audio files
1. Search for all single bat audio files with a duration that is 90-110% of $multibat_{duration}$ 
1. Choose $N$ files from this pool of audio files (where $N$ is either 2 (pair) or 3(triplet))

    1. Check to see if the chosen files have already been used to make a virtual multi bat file (see Step 4)
    
    1. If they have go to step 3 and choose again
    
    1. If after 100,000 tries the same files are coming up, then skip finding a matching audio file for this particular multi bat file.

1. Store the names of the single bat audio files used to make virtual multi bat files. 

1. Add the waveforms of the chosen $N$ files to create a virtual audio file. 


In [17]:
def make_virtual_multi_bat_audio(multi_bat_data, single_bat_files, max_tries = 100*10**3,
                                **kwargs):
    '''
    Makes multiple virtual audio files by adding duration matched audio files together based on 
    the number of bats that an observed audio file had. 
    
    All resulting virtual audio files are made of  unique file mixtures, ie. if there's a 
    virtual audio file made with A+B+C, then A,B,C is not used for any other virtual audio files
    in the coming iterations. There may be files with just AB or AC in the virtual audio file 
    mixtures. 
        
    Parameters
    ----------
    multi_bat_data: pd.DataFrame
        DataFrame with names of the multi bat audio files,
        the number of bats that the corresponding annotation 
        had, and the duration of the audio file. The compulsory
        columns are: 'file_name', 'num_bats', 'duration'
    single_bat_data:
        The names of files with single bat audio in them. The 
        compulsory columns are: 'file_name', 'duration', 'posix_start_time' and 'posix_end_time'
    time_interval : float>0
        The minimum time interval between single bat audio files in seconds. 
    max_tries : int, optional
        The maximum number of random samplings to try before giving up and moving onto the 
        next multi bat audio file. In the ideal case each virtual multi bat audio needs
        to be from a unique single bat audio file within the acceptable duration range, 
        *And* that hasn't been chosen before. Defaults to 100,000 trials.
    duration_width : 0<float <1, optional 
        When files are chosen that are of similar length to a multi bat file, they need to be within 
        1 +/- duration_width of the multi bat file's duration. 

    Returns
    -------
    matched_files: dictionary
        Keys contain the multi bat audio file name and the items contain 
        the duration matched single bat audio files in a list. 
    '''
    already_chosen = []
    duration_width = kwargs.get('duration_width', 0.1)
    time_interval = kwargs['time_interval']
    matched_files = {}
    
    all_singlebat_times = np.column_stack((single_bat_files['posix_start_time'],
                                           single_bat_files['posix_end_time']))
    
    for i, row in multi_bat_data.iterrows():
        filename, numbats, multibat_duration, _, _ = row
        
        interval_sep_inds = annots_by_time.generate_time_separated_folds(all_singlebat_times, 
                                                         time_interval,num_runs=1)
        interval_separated_single_bat_files = single_bat_files.loc[interval_sep_inds[0],:].reset_index(drop=True)
        valid_rows = np.logical_and(interval_separated_single_bat_files['duration']<(1+duration_width)*multibat_duration,
                                   interval_separated_single_bat_files['duration']>=(1-duration_width)*multibat_duration)
        valid_data = interval_separated_single_bat_files[valid_rows]
        
        if valid_data.shape[0] == 0:
            continue
        #print(valid_data)
        unique_files_not_found = True
        tries = 0
        while np.logical_and(unique_files_not_found, tries<max_tries):
            try:
                chosen_files = random.sample(valid_data['file_name'].to_list(), k=numbats)
            except:
                print('Error:',valid_data['file_name'],numbats)
            chosen_file_set = frozenset(chosen_files)
            overlap = len(chosen_file_set.intersection(frozenset(already_chosen)))
            if overlap <1:
                unique_files_not_found = False
                matched_files[filename] = chosen_files
                already_chosen.append(chosen_file_set)
                tries = 0
            else:
                tries += 1
            if tries>=max_tries:
                print(f'Unable to find correct matches for {filename}')
    
    return matched_files

In [18]:
q = frozenset(['a','b'])
r = frozenset(['a','c'])

In [19]:
q.intersection(r)

frozenset({'a'})

In [20]:
from random import sample

In [21]:
audio_folder = '../individual_call_analysis/hp_annotation_audio/'

def make_fileinfo_dataframe(df, audio_folder):
    '''
    DataFrame with at least the following columns: 'file_name', 'num_bats',
    'posix_end_time'
    
    '''

    file_durations = []
    num_bats = []
    all_file_names = []
    posix_end_time = []
    posix_start_time = []
    for file_name in tqdm.tqdm(np.unique(df['file_name'])):        

        file_path = cca.find_file_in_folder(file_name, audio_folder)
        file_durations.append(sf.info(file_path[0]).duration)

        one_annot = df[df['file_name']==file_name]
        num_bats.append(int(np.unique(one_annot['num_bats'])))
        ind = one_annot.index[0]
        posix_end_time.append(one_annot.loc[ind,'posix_end_time'])
        posix_start_time.append(one_annot.loc[ind,'posix_start_time'])
        all_file_names.append(file_name)

    df_fileinfo = pd.DataFrame(data={'file_name':all_file_names,
                                           'num_bats':num_bats,
                                           'duration':file_durations,
                                    'posix_end_time':posix_end_time,
                                    'posix_start_time':posix_start_time})
    return df_fileinfo

In [22]:
for each in [multibats, singles]:
    each['file_name'] = each.apply(lambda X: X['valid_annotations']+'_hp.WAV',1)

In [23]:
multibat_fileinfo = make_fileinfo_dataframe(multibats, audio_folder)
singlebat_fileinfo = make_fileinfo_dataframe(singles, audio_folder)

  0%|                                                                                          | 0/112 [00:00<?, ?it/s]

  6%|█████▏                                                                            | 7/112 [00:00<00:01, 69.81it/s]

Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!


 12%|██████████▏                                                                      | 14/112 [00:00<00:01, 69.81it/s]

 19%|███████████████▏                                                                 | 21/112 [00:00<00:01, 69.82it/s]

...

 97%|█████████████████████████████████████████████████████████████████████████████▉  | 294/302 [00:04<00:00, 67.62it/s]

Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!


100%|███████████████████████████████████████████████████████████████████████████████▋| 301/302 [00:04<00:00, 68.27it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 302/302 [00:04<00:00, 66.48it/s]

Match found!


In [24]:
# repeat the matching exercise N times and choose the run with most number of unique file mixtures
multibat_matched_runs = {}
for i in tqdm.trange(25):
    multibat_matched_runs[i] = make_virtual_multi_bat_audio(multibat_fileinfo, singlebat_fileinfo, duration_width=0.2,
                                                       time_interval=300)

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

Error: 1    matching_annotaudio_Aditya_2018-08-17_12_4_hp.WAV
Name: file_name, dtype: object 2
Error: 1    matching_annotaudio_Aditya_2018-08-17_12_4_hp.WAV
Name: file_name, dtype: object 2
Error: 9    matching_annotaudio_Aditya_2018-08-19_23_6_hp.WAV
Name: file_name, dtype: object 2
Error: 19    matching_annotaudio_Aditya_2018-08-16_21502300...
36    matching_annotaudio_Aditya_2018-08-17_34_18_hp...
Name: file_name, dtype: object 3
Error: 10    matching_annotaudio_Aditya_2018-08-19_23_6_hp.WAV
Name: file_name, dtype: object 3


Error: 3    matching_annotaudio_Aditya_2018-08-17_12_4_hp.WAV
Name: file_name, dtype: object 2
Error: 9    matching_annotaudio_Aditya_2018-08-19_23_6_hp.WAV
Name: file_name, dtype: object 3


  4%|███▎                                                                               | 1/25 [00:00<00:05,  4.20it/s]

Error: 10    matching_annotaudio_Aditya_2018-08-19_23_6_hp.WAV
Name: file_name, dtype: object 3
Error: 3    matching_annotaudio_Aditya_2018-08-17_12_4_hp.WAV
Name: file_name, dtype: object 2
Error: 3    matching_annotaudio_Aditya_2018-08-17_12_4_hp.WAV
Name: file_name, dtype: object 2
Error: 10    matching_annotaudio_Aditya_2018-08-19_23_6_hp.WAV
Name: file_name, dtype: object 2


...

 96%|██████████████████████████████████████████████████████████████████████████████▋   | 24/25 [00:06<00:00,  3.91it/s]

Error: 3    matching_annotaudio_Aditya_2018-08-17_12_4_hp.WAV
Name: file_name, dtype: object 2
Error: 17    matching_annotaudio_Aditya_2018-08-16_21502300...
Name: file_name, dtype: object 2
Error: 9    matching_annotaudio_Aditya_2018-08-19_23_6_hp.WAV
Name: file_name, dtype: object 3
Error: 9    matching_annotaudio_Aditya_2018-08-19_23_6_hp.WAV
Name: file_name, dtype: object 3
Error: 1    matching_annotaudio_Aditya_2018-08-17_12_4_hp.WAV
Name: file_name, dtype: object 2
Error: 1    matching_annotaudio_Aditya_2018-08-17_12_4_hp.WAV
Name: file_name, dtype: object 2
Error: 10    matching_annotaudio_Aditya_2018-08-19_23_6_hp.WAV
Name: file_name, dtype: object 2
Error: 17    matching_annotaudio_Aditya_2018-08-16_21502300...
36    matching_annotaudio_Aditya_2018-08-17_34_18_hp...
Name: file_name, dtype: object 3
Error:

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:06<00:00,  3.99it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:06<00:00,  3.93it/s]

 9    matching_annotaudio_Aditya_2018-08-19_23_6_hp.WAV
Name: file_name, dtype: object 3
Error: 1    matching_annotaudio_Aditya_2018-08-17_12_4_hp.WAV
Name: file_name, dtype: object 2
Error: 10    matching_annotaudio_Aditya_2018-08-19_23_6_hp.WAV
Name: file_name, dtype: object 3
Error: 9    matching_annotaudio_Aditya_2018-08-19_23_6_hp.WAV
Name: file_name, dtype: object 3


In [25]:
 multibat_fileinfo.loc[0,:]

file_name           matching_annotaudio_Aditya_2018-08-16_21502300...
num_bats                                                            2
duration                                                         0.76
posix_end_time                                            1.53445e+09
posix_start_time                                          1.53445e+09
Name: 0, dtype: object

In [26]:
num_unique_filemixes = []
for i, each in multibat_matched_runs.items():
    filemix_set = [frozenset(filemixes) for keys, filemixes in each.items()]
    unique_filemixes = len(frozenset(filemix_set))
    print(i, unique_filemixes)
    num_unique_filemixes.append(unique_filemixes)

0 81
1 84
2 81
3 80
4 79
5 81
6 79
7 82
8 79
9 85
10 84
11 82
12 82
13 81
14 81
15 84
16 78
17 83
18 81
19 80
20 83
21 85
22 80
23 83
24 81


In [27]:
multibat_matched = multibat_matched_runs[int(np.argmax(num_unique_filemixes))]

There are some file mixes that are repeated - let's get rid of those! We'll do this by removing the second repeated file mix.

In [28]:
multibat_matched_unique = {}

filemixes_tillnow = []
for obsfile, filemix in multibat_matched.items():
   
    already_encountered = filemix in filemixes_tillnow
    if not already_encountered:
        multibat_matched_unique[obsfile] = filemix
        filemixes_tillnow.append(filemix)


In [29]:
len(multibat_matched)

104

In [30]:
len(multibat_matched_unique)

89

In [31]:
virtual_audio_folder = 'virtual_multi_bat_audio/'
if not os.path.isdir(virtual_audio_folder):
    os.mkdir(virtual_audio_folder)
else:
    # delete all files in the folder if it already exists
    files = glob.glob(virtual_audio_folder+'*')
    for f in files:
        os.remove(f)

for multibatfile, matched_files in multibat_matched_unique.items():
    # load the 1st channel of all the audio files
    audio_data = []
    for each in matched_files:
        filepath = cca.find_file_in_folder(each, audio_folder)[0]
        audio, fs = sf.read(filepath)
        audio_data.append(audio[:,0])
    smallest_audio = np.min([each.size for each in audio_data])
    combined_audio = np.zeros(smallest_audio)
    
    for singleaudio in audio_data:
        combined_audio += singleaudio[:smallest_audio]
    final_path = os.path.join(virtual_audio_folder,multibatfile[:-4]+'_singlebatmixed.WAV')
    sf.write(final_path, combined_audio, fs)
        

Match found!
Match found!
Match found!
Match found!


Match found!
Match found!
Match found!
Match found!


Match found!
Match found!
Match found!
Match found!


Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!


Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!


...

Match found!
Match found!
Match found!
Match found!


Match found!
Match found!
Match found!


Match found!
Match found!
Match found!
Match found!
Match found!
Match found!
Match found!


Match found!
Match found!
Match found!
Match found!
Match found!


Match found!
Match found!
Match found!
Match found!


### Also save the source file information for all the virtual multi bat audio files


In [32]:
obs_multibat_files = list(multibat_matched.keys())
source_single_files = [ str(each[1])[2:-2] for each in list(multibat_matched.items())]

virtual_multibat_fileaddition = pd.DataFrame(data={'multibat_filename': obs_multibat_files,
                                                  'source_single_bat_files': source_single_files})

def fix_filenaming(each):
    part1, part2 = each.split('.')
    actual_name = part1+'_singlebatmixed.'+part2
    return actual_name

virtual_multibat_fileaddition['actual_virtual_filenames'] = virtual_multibat_fileaddition['multibat_filename'].apply(fix_filenaming)

virtual_multibat_fileaddition.to_csv('source_files_for_virtual_multibat_audio.csv')

In [33]:
plt.figure()
ax1 = plt.subplot(211)
plt.plot(np.linspace(0,combined_audio.size/fs, combined_audio.size),combined_audio)
ax2 = plt.subplot(212, sharex=ax1)
plt.specgram(combined_audio, Fs=fs);


<IPython.core.display.Javascript object>

(array([[6.33347740e-13, 5.49569677e-13, 6.25005606e-13, ...,
         6.41614927e-13, 6.70312101e-13, 6.07532601e-13],
        [3.21480324e-13, 2.60976449e-13, 3.20224333e-13, ...,
         3.45458214e-13, 3.64432201e-13, 3.06510023e-13],
        [1.60978964e-15, 1.70651796e-16, 1.62446614e-17, ...,
         3.03553631e-15, 2.35936951e-15, 1.10300262e-15],
        ...,
        [4.28985321e-11, 4.57509024e-11, 4.31093045e-12, ...,
         1.86572874e-11, 3.68712076e-11, 1.32492244e-10],
        [2.14719004e-11, 1.72138147e-10, 5.50819513e-11, ...,
         4.74158710e-11, 1.56276279e-13, 1.22874210e-10],
        [2.18696143e-12, 8.60661887e-11, 8.01223250e-12, ...,
         1.71916082e-11, 8.88742753e-12, 7.28680057e-11]]),
 array([     0.    ,    976.5625,   1953.125 ,   2929.6875,   3906.25  ,
          4882.8125,   5859.375 ,   6835.9375,   7812.5   ,   8789.0625,
          9765.625 ,  10742.1875,  11718.75  ,  12695.3125,  13671.875 ,
         14648.4375,  15625.    ,  16601.5625,

In [34]:
print(f'Notebook run ended at {dt.datetime.now()}')

Notebook run ended at 2020-12-17 15:16:35.611107
